In [1]:
import re
from pymorphy2 import MorphAnalyzer
from natasha import Segmenter, Doc, NewsMorphTagger, NewsEmbedding
from pymystem3 import Mystem
from nltk import word_tokenize
from nltk import ngrams

### Задание 1

Я выбрала в качестве сложных слов:
1. Слова, которые считаются окказиональными, появляющимися в обычной речи, но считающимися ошибочными (поуезжать, пробудясь, принесть и тд). Такие формы скорее не находятся в словаре и могли не встречаться модели.
2. Формы, которые омонимичны, но относятся к разным частям речи в зависимости от контекста (светило, печь, мыла). Такие формы интересны с точки зрения понимания модели контекста предложения (но это относится только к моделям, определяющим контекст)
3. Слова с неизвестными корнями, но форму которых можно понять по морфологическим признакам (зафрендить, бомжатник). Аналогично первой группе слов, в данном случае слова скорее неизвестны модели.
4. Аббревиатуры. Они тоже могут не быть в словаре модели и, в отличие от п.1 и 3, не имеют морфологических характеристик, которые могут помочь предсказать часть речи. 

Так как тэгсеты различаются, я решила свести все возможные метки к наиболее узкому сету: NOUN, VERB, ADJ, ADV, GRND (деепричастие), PRT (причастие), ADP (предлог), PART (частица), CONJ, NUM, PRON. Перевод из одного формата в другой записан в ячейке ниже. Я решила объединить краткие и полные формы прилагательных под одним тэгом, так как важнее, чтобы модель угадала, что это вообще прилагательное, когда как краткость и полнота скорее второстепенная характеристика, аналогично с разными видами местоимений и причастий. Вспомогательный глагол я относила к общему классу глаголов по той же причине. 

In [2]:
tag_to_tag = {
    'SPRO': 'PRON', 'ADVPRO': 'ADV', 'APRO': 'PRON',
    'NPRO': 'PRON', 'DET': 'PRON', 'S': 'NOUN',
    'PROPN': 'NOUN', 'V': 'VERB', 'INFN': 'VERB',
    'PRTS': 'PRT', 'PRTF': 'PRT', 'ADVB': 'ADV',
    'ANUM': 'NUM', 'NUMR': 'NUM', 'ADJF': 'ADJ',
    'ADJS': 'ADJ', 'PR': 'ADP', 'CCONJ': 'CONJ',
    'SCONJ': 'CONJ', 'PRCL': 'PART', 'AUX': 'VERB',
    'PREP': 'ADP', 'A': 'ADJ'
}

### Задание 2

Я выбрала теггеры pymorphy, natasha и mystem. 
Я сделала словарь, в котором ключ - это предложение, а значение это вложенный словарь, в котором записаны отдельно токены, правильные части речи и предсказания каждой из исследуемых моделей. Конвертация предсказаний в теги ручной разметки происходит внутри функций.

In [3]:
sents = {}
re_sent = re.compile(r'#sent = (.+?)\n')
with open('morph.txt', 'r') as f:
    txt = f.readlines()

for line in txt:
    if line.startswith('#sent'):
        sent = re_sent.search(line).group(1)
        sents[sent] = {'words': [], 'pos': []}
    elif line != '\n':
        word, pos = line.split()
        sents[sent]['words'].append(word.strip())
        sents[sent]['pos'].append(pos)

In [6]:
word_cnt = 0
for v in sents.values():
    word_cnt += len(v['words'])
print(word_cnt)

204


In [4]:
morph = MorphAnalyzer()
def pymorhy_tag(sents):
    for k, v in sents.items():
        poses = []
        for word in v['words']:
            p = morph.parse(word)
            pos = p[0].tag.POS
            if pos in tag_to_tag.keys():
                poses.append(tag_to_tag[pos])
            else:
                poses.append(pos)
        sents[k]['pymorphy'] = poses

In [5]:
def natasha_tag(sents):
    segmenter = Segmenter()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    for k, v in sents.items():
        poses = []
        doc = Doc(k)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for token in doc.tokens:
            pos = token.pos
            if pos == 'VERB':
                if token.feats['VerbForm'] == 'Conv':
                    pos = 'GRND'
                elif token.feats['VerbForm'] == 'Part':
                    pos = 'PRT'
            if pos in ['ADJ', 'ADV']:
                if token.feats['Degree'] == 'Cmp':
                    pos = 'COMP'
            if pos != 'PUNCT':
                if pos in tag_to_tag.keys():
                    poses.append(tag_to_tag[pos])
                else:
                    poses.append(pos)
        sents[k]['natasha'] = poses 

In [6]:
def mystem_tag(sents):
    m = Mystem()
    for k, v in sents.items():
        poses = []
        analysis = m.analyze(k)
        for word in analysis:
            if word['text'].isalpha():
                feats = word['analysis'][0]['gr']
                pos = feats.split('=')[0].split(',')[0]
                if pos == 'V':
                    if 'деепр' in feats:
                        pos = 'GRND'
                    elif 'прич' in feats:
                        pos = 'PRT'
                if pos in ['A', 'ADV']:
                    if 'срав' in feats:
                        pos = 'COMP'
                if pos in tag_to_tag.keys():
                    poses.append(tag_to_tag[pos])
                else:
                    poses.append(pos)
        sents[k]['mystem'] = poses

In [10]:
pymorhy_tag(sents)
natasha_tag(sents)
mystem_tag(sents)

### Задание 3

In [11]:
def count_accuracy(tagger_name):
    acc_count = 0
    all_pos = 0
    errors = []
    for v in sents.values():
        for y_true, y_pred, word in zip(v['pos'], v[tagger_name], v['words']):
            if y_true == str(y_pred):
                acc_count += 1
            else:
                errors.append([word, y_true, y_pred])
            all_pos += 1
    return acc_count / all_pos, errors

In [12]:
py_acc, py_errs = count_accuracy('pymorphy')
nat_acc, nat_errs = count_accuracy('natasha')
my_acc, my_errs = count_accuracy('mystem')

print('pymorphy:', py_acc)
print('natasha:', nat_acc)
print('mystem:', my_acc)

pymorphy: 0.8676470588235294
natasha: 0.8078817733990148
mystem: 0.9458128078817734


Лучшим теггером стал mystem!

### Задание 4

На вход чанкеру подаётся шаблон, по которому нужно искать н-грамму, и предложение. Предложение делится на леммы и части речи, а после составляется список, в котором один элемент - это кортеж ('лемма', 'часть речи'). Из этого списка составляются биграммы и ищется нужный паттерн. Если паттерн найден, то берётся индекс биграммы и выводится сочетание лемм, соотвествующее шаблону.

In [20]:
is_cyrilic = re.compile(r'[а-яА-ЯёЁ]+?\b')
def mystem_tag(sent):
    m = Mystem()
    poses = []
    analysis = m.analyze(sent)
    for word in analysis:
        if is_cyrilic.match(word['text']):
            try:
                feats = word['analysis'][0]['gr']
                pos = feats.split('=')[0].split(',')[0]
            except:
                pos = 'X'
            if pos == 'V':
                if 'деепр' in feats:
                    pos = 'GRND'
                elif 'прич' in feats:
                    pos = 'PRT'
            if pos in ['A', 'ADV']:
                if 'срав' in feats:
                    pos = 'COMP'
            if pos in tag_to_tag.keys():
                poses.append(tag_to_tag[pos])
            else:
                poses.append(pos)
    return poses

In [21]:
patterns = ['грязный+NOUN', 'ADJ+NOUN', 'NOUN+быть+ADJ']
def chunker(pattern, text):
    m = Mystem()
    result = []
    pattern = pattern.split('+')
    lemmas = [lemma for lemma in m.lemmatize(text) if is_cyrilic.match(lemma)]
    poses = mystem_tag(text)
    result_sentence = [(l, p) for l, p in zip(lemmas, poses)]
    all_ngrams = list(ngrams(result_sentence, len(pattern)))
    word_ngrams = list(ngrams(lemmas, len(pattern)))
    for j, ngram in enumerate(all_ngrams):
        cnt = 0
        for i, word in enumerate(pattern):
            if word in ngram[i]:
                cnt += 1
        if cnt == len(pattern):
            result.append(' '.join(word_ngrams[j]))
    return result

In [22]:
chunker('ADJ+NOUN', 'Я хотела розовую куклу')

['розовый кукла']